In [1]:
# Install required libraries

!pip install openai==0.28
!pip install transformers datasets nltk openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
import torch
import nltk
from nltk.translate.gleu_score import sentence_gleu
import openai

# Downloading NLTK data
nltk.download("punkt")

# Setting up OpenAI API key that is required for feedback generation
openai.api_key = 'insert-your-api-key'


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Loading the JFLEG dataset
dataset = load_dataset("jhu-clsp/jfleg")

# Splitting the dataset into train, validation, and test sets
train_test_split = dataset['validation'].train_test_split(test_size=0.2)
dataset = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test'],
    'test': dataset['test']
})

# Loading T5 model and tokenizer
model_name = "t5-base"
model = T5ForConditionalGeneration.from_pretrained(model_name).to("cuda")
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Preprocessing data for model input
def preprocess_function(examples):
    inputs = ["fix: " + sentence for sentence in examples["sentence"]]
    targets = [correction[0] if correction else "" for correction in examples["corrections"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = tokenizer(text_target=targets, max_length=128, truncation=True, padding="max_length")["input_ids"]
    return model_inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.94k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/141k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/755 [00:00<?, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/604 [00:00<?, ? examples/s]

Map:   0%|          | 0/151 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

In [4]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./t5_finetuned_jfleg",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="none"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# Initializing Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Fine-tuning the model
trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.125878
2,No log,0.119956
3,No log,0.121164
4,No log,0.120590
5,No log,0.123461
6,No log,0.124217
7,0.080800,0.125719
8,0.080800,0.126802
9,0.080800,0.127886
10,0.080800,0.128024


TrainOutput(global_step=760, training_loss=0.07180208908884149, metrics={'train_runtime': 570.9681, 'train_samples_per_second': 10.579, 'train_steps_per_second': 1.331, 'total_flos': 919526414745600.0, 'train_loss': 0.07180208908884149, 'epoch': 10.0})

In [13]:
# Function for grammar correction
def correct_grammar(test_data):
    corrected_sentences = []
    for sentence in test_data["sentence"]:
        input_text = f"fix: {sentence}"
        input_ids = tokenizer.encode(input_text, return_tensors="pt").to("cuda")
        outputs = model.generate(input_ids, max_length=64, num_beams=4, early_stopping=True)
        corrected_sentence = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Keeping original sentence if corrected output is empty
        corrected_sentence = corrected_sentence if corrected_sentence.strip() else sentence
        corrected_sentences.append(corrected_sentence)

        print(f"Original: {sentence}")
        print(f"Corrected: {corrected_sentence}")
        print("="*40)

    return corrected_sentences



In [14]:
# Function to evaluate GLEU score
def evaluate_gleu(corrected_sentences, test_data):
    gleu_scores = []
    for corrected_sentence, references in zip(corrected_sentences, test_data["corrections"]):
        prediction_tokens = corrected_sentence.split()
        reference_tokens = [ref.split() for ref in references]
        gleu_score = sentence_gleu(reference_tokens, prediction_tokens)
        gleu_scores.append(gleu_score)

    avg_gleu_score = sum(gleu_scores) / len(gleu_scores)
    print(f"\nAverage GLEU Score: {avg_gleu_score:.4f}")
    return avg_gleu_score

In [16]:
import openai

# Function to provide feedback using OpenAI API
def provide_feedback(sentences):
    for sentence in sentences[:10]:  # Automatically limits to the first 10 sentences
        prompt = f"The sentence is: '{sentence}'\n" \
                 f"Please provide detailed feedback on the grammatical correctness of this sentence and suggest improvements if necessary."
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a grammar expert."},
                {"role": "user", "content": prompt}
            ]
        )
        feedback = response['choices'][0]['message']['content']
        print(f"Feedback for: '{sentence}'\n{feedback}")
        print("="*80)

# Example usage with any list of sentences
sentences = [...]  # Replace with your full list of sentences




In [17]:
corrected_sentences = correct_grammar(tokenized_datasets["test"])

# Evaluating model based on GLEU score on the test set
evaluate_gleu(corrected_sentences, tokenized_datasets["test"])

provide_feedback(corrected_sentences)


Original: New and new technology has been introduced to the society .
Corrected: New and new technology has been introduced to the society.
Original: One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countries will outweigh any rise in motorization levels in the poorer countries .
Corrected: One possible outcome is that an environmentally-induced reduction in motorization levels in the richer countries will outweigh any rise in motorization levels in the poorer countries.
Original: Every person needs to know a bit about math , sciences , arts , literature and history in order to stand out in society .
Corrected: Every person needs to know a bit about math, sciences, arts, literature and history in order to stand out in society.
Original: While the travel company will most likely show them some interesting sites in order for their customers to advertise for their company to their family and friends , it is highly unlikely , that the c